# The Unique Properties of Qubits

*This workshop covers topics from the [The Unique Properties of Qubits]() section of the textbook [Learn Quantum Computation using Qiskit](https://community.qiskit.org).*

We'll now go back to the first program of the workshop [The Atoms of Computation](Atoms-Computation.ipynb) and add some additional features.

A big difference between qubits and normal bits is that there is only one way to get an output from a normal bit: You just look at it, and see if it is `0` or `1`. For a qubit, however, there are multiple ways you can choose to do it. One is the method applied when you use a simple `measure` operation. This is called a z measurement.

```python
# z measurement of qubit j
qc.measure(j,j)
```

Another form of measurement, known as an x measurement, cannot be performed quite as directly in Qiskit. Instead, it needs two lines.

```python
# x measurement of qubit j
qc.h(j)
qc.measure(j,j)
```

In the following we expand upon the `meas` variable introduced in the last workshop. There it held just a single circuit. Here it will hold four: a single qubit measurement for each qubit, and for each basis. It is constructed as follows.

```python
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
```

For example, `meas[0]['X']` contains the circuit that measures qubit 0 in the x basis.

```python
meas[0]['X'] = QuantumCircuit(2,2)
meas[0]['X'].h(j)
meas[0]['Z'].measure(j,j)
```

The reason why these single qubit measurements are defined in two qubit circuits is simply so that they can be easily added together. For example, the circuit with a z measurement on qubit 0 and an x measurement on qubit 1 corresponds to the circuit `meas[0]['Z']+meas[1]['X']`.

The choice as to which measurement type to use will be controlled by the variable `basis`. The name is chosen because the type of measurement is also known as the measurement basis. We will set `basis[0]` will be `'Z'` or `'X'` depending on whether qubit 0 should be measured in basis z or x, respectively, and similarly for `basis[1]` and qubit 1. This means we can access the circuit for the measurement of qubit `j` with `meas[j][basis[j]]`.

To make it obvious which basis is being used, we will display the results in different places. The output of a z measurement of qubit 1 will be displayed on pixel (1,2), as before. The output of an x measurement will instead be displayed below this at (1,4). Both will have square of dim pixels around them. And both will also be set to dim when their basis is not in use.

To choose which basis to use for each qubit, we'll use the buttons X and O. The X button will toggle between the two bases for qubit 1, and the O button will do the same for qubit 0.

In [ ]:
%matplotlib notebook

In [ ]:
import pew # setting up tools for the pewpew
from microqiskit import QuantumCircuit, simulate # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys&pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = simulate(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

In the program above, it is as if each qubit is hiding in a box. If we peek into the hole at the top, we see its z basis information. For the hole at the bottom, we see the x basis information.

Actually, we are seeing many qubits, since a new pair is generated and simulated every frame. But since each pair is created in the same state, it gives us a glimpse into the life of a qubit.

To get even more of an insight, let's add the ability to change the state that is being simulated. We'll do this by restoring some of the lines we had before, which added `x` gates into `qc` when ▲ and ◀︎ are pressed. Similarly, we can add an `h` to `qc` when ▼ and ► are pressed.

In [ ]:
import pew # setting up tools for the pewpew
from microqiskit import QuantumCircuit, simulate # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys&pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
        if keys&pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys&pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
        if keys&pew.K_RIGHT:
            qc.h(0) # h for qubit 0 when RIGHT is pressed
        if keys&pew.K_DOWN:
            qc.h(1) # h for qubit 1 when DOWN is pressed
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = simulate(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

In [ ]:
import pew # setting up tools for the pewpew
from microqiskit import QuantumCircuit, simulate # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
qc.h(0)
qc.cx(0,1)

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys==pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys==pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
        if keys==pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys==pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
        if keys==pew.K_RIGHT:
            qc.h(0) # h for qubit 0 when RIGHT is pressed
        if keys==pew.K_DOWN:
            qc.h(1) # h for qubit 1 when DOWN is pressed
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = simulate(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)
        
    # flag up the '11' output by lighting up pixels in the bottom-center
    if m[0]=='11':
        screen.pixel(3,6,3)
        screen.pixel(4,6,3)
    else:
        screen.pixel(3,6,0)
        screen.pixel(4,6,0)
    # same with '00' and the top center
    if m[0]=='00':
        screen.pixel(3,0,3)
        screen.pixel(4,0,3)
    else:
        screen.pixel(3,0,0)
        screen.pixel(4,0,0)  

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

From what we've seen so far, you'd be forgiven for thinking that qubits are just bits with some randomness bundled in. However, they are more complex than that. This becomes evident if we try to simulate them, since simulating a collection of $n$ qubits requires us to keep track of at least $2^n$ complex numbers. For around 50 qubits, this is a task that you'd need the world's biggest supercomputers for.

The reason that simulation is so difficult is that it needs to account for types of correlation that are not possible with classical bits. One example of this is explained in the [The Unique Properties of Qubits]() section of the [Learn Quantum Computation using Qiskit](https://community.qiskit.org) textbook. There a two qubit state is used for which z measurements of both qubits will never output `'00'`, but will output the other three possibilities with equal probability.

Let's create this state here, so that we can explore it on the PewPew. We'll take a different approach to setting it up than was used in the textbook. Specifically, we'll describe the state with a so-called *state vector*, and then load this into the circuit. These vectors are a list with one entry for each possible output. For two qubits, this means a list of four elements, for `'00'`, `'01'`, `'10'` and `'11'`, respectively.

These elements are the square roots of the probabilities for each outcome (when both qubits are measured in the z basis). Since the non-zero probabilties should be $1/3$, the corresponding element should be $0.57735$. So the state vector will be

```python
[0,0.57735,0.57735,0.57735]
```

Using the `initialize` method of a `QuantumCircuit` object, we can then set this state up

```python
qc.initialize([0,0.57735,0.57735,0.57735])
```

In [ ]:
import pew # setting up tools for the pewpew
from microqiskit import QuantumCircuit, simulate # setting up tools for quantum

pew.init() # initialize the game engine...
screen = pew.Pix() # ...and the screen

qc = QuantumCircuit(2,2) # create an empty circuit with two qubits and two output bits
qc.initialize([0,0.57735,0.57735,0.57735])

# create circuits with the required measurements, so we can add them in easily
meas = [{},{}]
for j in range(2):
    meas[j]['Z'] = QuantumCircuit(2,2)
    meas[j]['Z'].measure(j,j)
    meas[j]['X'] = QuantumCircuit(2,2)
    meas[j]['X'].h(j)
    meas[j]['X'].measure(j,j)
    
basis = ['Z','Z'] # set the initial measurement basis for each qubit

# loop over the squares centered on (1,2), (6,2) (1,4) and (6,4) and make all dim
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    for dX in [+1,0,-1]:
        for dY in [+1,0,-1]:
            screen.pixel(X+dX,Y+dY,2)
pew.show(screen)
        
for (X,Y) in [(1,2),(6,2),(1,4),(6,4)]:
    screen.pixel(X,Y,0) # turn off the center pixels of the squares  

old_keys = 0
while True: # loop which checks for user input and responds

    # look for and act upon key presses
    keys = pew.keys() # get current key presses
    if keys!=0 and keys!=old_keys:
        if keys&pew.K_O:
            basis[0] = 'X'*(basis[0]=='Z') + 'Z'*(basis[0]=='X') # toggle basis for right qubit
        if keys&pew.K_X:
            basis[1] = 'X'*(basis[1]=='Z') + 'Z'*(basis[1]=='X') # toggle basis for left qubit
        if keys&pew.K_UP:
            qc.x(0) # x for qubit 0 when UP is pressed
        if keys&pew.K_LEFT:
            qc.x(1) # x for qubit 1 when LEFT is pressed
        if keys&pew.K_RIGHT:
            qc.h(0) # h for qubit 0 when RIGHT is pressed
        if keys&pew.K_DOWN:
            qc.h(1) # h for qubit 1 when DOWN is pressed
    old_keys = keys 
    
    # execute the circuit and get a single sample of memory for the given measurement bases
    m = simulate(qc+meas[0][basis[0]]+meas[1][basis[1]],shots=1,get='memory')
    
    # turn the pixels (1,2) and (1,4) (depending on basis[1]) on or off (depending on m[0][0])
    if m[0][0]=='1':
        if basis[1]=='Z':
            screen.pixel(1,2,3)
        else:
            screen.pixel(1,4,3)
    else:
        if basis[1]=='Z':
            screen.pixel(1,2,0)
        else:
            screen.pixel(1,4,0)
    # do the same for pixels (6,2) and (6,4)
    if m[0][1]=='1':
        if basis[0]=='Z':
            screen.pixel(6,2,3)
        else:
            screen.pixel(6,4,3)
    else:
        if basis[0]=='Z':
            screen.pixel(6,2,0)
        else:
            screen.pixel(6,4,0)
            
    # turn the pixels not used to display m[0] to dim
    if basis[1]=='Z':
        screen.pixel(1,4,2)
    else:
        screen.pixel(1,2,2)
    if basis[0]=='Z':
        screen.pixel(6,4,2)
    else:
        screen.pixel(6,2,2)
        
    # flag up the '11' output by lighting up pixels in the bottom-center
    if m[0]=='11':
        screen.pixel(3,6,3)
        screen.pixel(4,6,3)
    else:
        screen.pixel(3,6,0)
        screen.pixel(4,6,0)
    # same with '00' and the top center
    if m[0]=='00':
        screen.pixel(3,0,3)
        screen.pixel(4,0,3)
    else:
        screen.pixel(3,0,0)
        screen.pixel(4,0,0)  

    pew.show(screen) # update screen to display the above changes

    pew.tick(1/6) # pause for a sixth of a second

Since the textbook already explains why this state is so unique, we don't need to repeat it here. But with the PewPew you'll be able to see the correlations by switching between z and x measurements, without needing to look at any histograms!

**[Click here for the next notebook](More-MicroQiskit.ipynb)**